In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
import  optuna
warnings.filterwarnings('ignore') #경고 메세지 창을 무시하기 
train_dir = './data/cat_dog_small/train'
validation_dir = './data/cat_dog_small/validation'
test_dir = './data/cat_dog_small/test'

train_datagen=ImageDataGenerator(rescale=1/255,
                                rotation_range=30,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=False,
                                fill_mode='nearest')
validation_datagen=ImageDataGenerator(rescale=1/255)

test_datagen=ImageDataGenerator(rescale=1/255,
                                rotation_range=30,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=False,
                                fill_mode='nearest')

train_generator=train_datagen.flow_from_directory(train_dir,
                                                 classes=['cats','dogs'],
                                                 target_size=(150,150),
                                                 batch_size=20,
                                                 class_mode='binary')

validation_generator=validation_datagen.flow_from_directory(validation_dir,
                                                 classes=['cats','dogs'],
                                                 target_size=(150,150),
                                                 batch_size=20,
                                                 class_mode='binary')
test_generator=test_datagen.flow_from_directory(test_dir,
                                                 classes=['cats','dogs'],
                                                 target_size=(150,150),
                                                 batch_size=20,
                                                 class_mode='binary')

In [9]:
model_base=VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150,150,3))
model_base.trainable=False
model=Sequential()
model.add(model_base)
model.add(Flatten())

model.add(Dense(units=128))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(rate=0.2))

model.add(Dense(units=1, activation='sigmoid'))


model.compile(optimizer=Adam(learning_rate=1e-2),
              loss='binary_crossentropy',
              metrics=['accuracy'])

call =EarlyStopping(patience=5,
                     monitor='val_loss',
                     verbose=1,
                     restore_best_weights=True)

check=ModelCheckpoint(filepath='./e.weights.h5',
                     save_best_only=True,
                     save_weights_only=True,
                      monitor='vall_accuracy',
                      verbose=1)
model.fit(train_generator,
          steps_per_epoch=100,
          epochs=30,
          validation_data=validation_generator,
          validation_steps=50,
          callbacks=[call, check],
          verbose=1)


Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 66s 655ms/step - accuracy: 0.7683 - loss: 0.5238 - val_accuracy: 0.8040 - val_loss: 0.4675
Epoch 2/30
 66/100 ━━━━━━━━━━━━━━━━━━━━ 15s 455ms/step - accuracy: 0.8269 - loss: 0.3786

KeyboardInterrupt: 

In [62]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_14 (Flatten)        (None, 8192)              0         
                                                                 
 dense_23 (Dense)            (None, 128)               1048704   
                                                                 
 batch_normalization_13 (Ba  (None, 128)               512       
 tchNormalization)                                               
                                                                 
 activation_12 (Activation)  (None, 128)               0         
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                     

In [ ]:

#fine tuning(미세 조정) 상위 filter부분의 이용해서 좀 더 나에게 맞는 모델을 만듬
#어떻게 얼마나 뒤에 filter을 손봐야 하는가 -> 대략 20% 정도 
#학습을 한 상태에서 이걸 하고 다시 모델을 학습함!

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os  #폴더경로 생성하기 위해서 사용함 !
from tensorflow.keras.preprocessing.image import ImageDataGenerator #입력데이터 처리
from tensorflow.keras.applications  import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization, Activation # activation하기 전에 드렁가야 함 !
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings #경고 메세지 창을 징귀
warnings.filterwarnings('ignore') #경고 메세지 창을 무시하기 
train_dir = './data/cat_dog_small/train'
validation_dir = './data/cat_dog_small/validation'
test_dir = './data/cat_dog_small/test'


#ImageDataGenerator 를 생성, 2개 필요 ->  어떤식으로 데이터를 가죠올 거니? 
#trian 1개 ->증식설정 필요/ Validation 1게 -> 증식 필요없음
train_datagen=ImageDataGenerator(rescale=1/255,
                                rotation_range=40,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest') # 변경하고 남은 부분을 근처와 비슷하게 채워라
                                # vertical_flit=True) # 위아래 위치를 바꿈 (사용 안하는거 추천!)  

validation_datagen=ImageDataGenerator(rescale=1/255)

#이미지 데이터 generator설정 잡기
train_generator =train_datagen.flow_from_directory(train_dir,
                                                  classes=['cats','dogs'],
                                                  target_size=(150,150),
                                                  batch_size=20,
                                                  class_mode='binary')
validation_generator=validation_datagen.flow_from_directory(validation_dir,
                                                           classes=['cats','dogs'],
                                                           target_size=(150,150),
                                                           batch_size=20,
                                                           class_mode='binary')


#model구현
model_base=VGG16(weights='imagenet',
                input_shape=(150,150,3),
                include_top=False)

#model_base에 대해서 False로 해서 filter값을 변경하지 않게 함(동결)
model_base_trainable=False
model=Sequential()
#hiddne layer
model.add(model_base) #내가 만든 model에 앞부분 filter에 model_base가 들어간다 이말임! -> 4차원
model.add(Flatten())  #위에 4차원을 1차원으로 만들어줌 ! : Flattne으 역활
model.add(Dense(units=128))
model.add(BatchNormalization()) #성능 향상을위해 사용함!
model.add(Activation('relu'))

model.add(Dropout(rate=0.2)) #무조건 activation두이 넣어야 함 !!! -> batchnomal이 있으니간 0.2정도만 
#output lyaer

model.add(Dense(units=1,
                activation='sigmoid'))



model.compile(optimizer=Adam(learning_rate=1e-4),
             loss='binary_crossentropy', 
             metrics=['accuracy'])

call=EarlyStopping(patience=5,
                  monitor='val_loss',
                  verbose=1,
                  restore_best_weights=True) #그떄까지 가장 좋은 값 return 

check=ModelCheckpoint(filepath='./f.weights.h5',
                     save_best_only=True,
                     save_weights_only=True,
                     monitor='val_accuracy',
                     verbose=1)

model.fit(train_generator,
          steps_per_epoch=100,
          epochs=30,
          validation_data=validation_generator,
          validation_steps=50,
          verbose=1,
          callbacks=[call])
                  
                  


In [ ]:
#fine_tuning
model_base.trainable=True
rate=len(model_base.layers)
fin_len=int(rate*0.7)

for layer in model_base.layers[:fin_len]:  
    layer.trainable = False 
for layer in model_base.layers[fin_len:]:  
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# model_base.summary()

# Model 학습(Fine Tuning)
model.fit(train_generator,
          steps_per_epoch=100,
          epochs=15,
          validation_data=validation_generator,
          validation_steps=50,
          callbacks=[call, check],
          verbose=1)

In [73]:
#EfficientNet 
!pip install tensorflow_hub
#이미 분류 / 전이 학습 / 영상 처리 및 분석 에서 사용함 
#고해상도 이미지 처리/ 제약된 계산 자원/ 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 55.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 65.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.13.0
    Uninstalling tensorboard-2.13.0:
      Successfully uninstalled tensorboard-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorflow
 

In [75]:
# import tensorflow_hub as hub
# model_base = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/google/efficientnetv2-xl-21k-ft1k/1",
#                                                trainable=True,
#                                                input_shape=(150,150,3)])
from tensorflow.keras.applications import EfficientNetB4
model_base= EfficientNetB4(weights='imagenet',
                           input_shape=(),
                           include_top=False)

In [88]:
print(tf.__version__)
from tensorflow.keras.applications import EfficientNetB4


2.13.0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os  #폴더경로 생성하기 위해서 사용함 !
from tensorflow.keras.preprocessing.image import ImageDataGenerator #입력데이터 처리
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings #경고 메세지 창을 징귀
warnings.filterwarnings('ignore') #경고 메세지 창을 무시하기 
train_dir = './data/cat_dog_small/train'
validation_dir = './data/cat_dog_small/validation'
test_dir = './data/cat_dog_small/test'


#ImageDataGenerator 를 생성, 2개 필요 ->  어떤식으로 데이터를 가죠올 거니? 
#trian 1개 ->증식설정 필요/ Validation 1게 -> 증식 필요없음
train_datagen=ImageDataGenerator(rescale=1/255,
                                rotation_range=40,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest') # 변경하고 남은 부분을 근처와 비슷하게 채워라
                                # vertical_flit=True) # 위아래 위치를 바꿈 (사용 안하는거 추천!)  

validation_datagen=ImageDataGenerator(rescale=1/255)

#이미지 데이터 generator설정 잡기
train_generator =train_datagen.flow_from_directory(train_dir,
                                                  classes=['cats','dogs'],
                                                  target_size=(150,150),
                                                  batch_size=20,
                                                  class_mode='binary')
validation_generator=validation_datagen.flow_from_directory(validation_dir,
                                                           classes=['cats','dogs'],
                                                           target_size=(150,150),
                                                           batch_size=20,
                                                           class_mode='binary')


#model구현


model_base = VGG16(weights='imagenet',
                   include_top=False,
                   input_shape=(150,150,3))

#model_base에 대해서 False로 해서 filter값을 변경하지 않게 함(동결)
model_base_trainable=False
model=Sequential()
#hiddne layer
model.add(model_base) #내가 만든 model에 앞부분 filter에 model_base가 들어간다 이말임! -> 4차원
model.add(Flatten())  #위에 4차원을 1차원으로 만들어줌 ! : Flattne으 역활
model.add(Dense(units=128))
model.add(BatchNormalization()) #성능 향상을위해 사용함!
model.add(Activation('relu'))

model.add(Dropout(rate=0.3)) #무조건 activation두이 넣어야 함 !!! -> batchnomal이 있으니간 0.2정도만 
#output lyaer

model.add(Dense(units=1,
                activation='sigmoid'))




init=1e-5
model.compile(optimizer=Adam(learning_rate=init),
              loss='binary_crossentropy',
              metrics=['accuracy'])

class LRFinderCallback(tf.keras.callbacks.Callback):
    def __init__(self, min_lr=1e-8, max_lr=10, steps=100):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_steps = steps
        self.history = {'lr': [],
                        'loss': []}
        
    def on_batch_end(self, batch, logs=None):
        lr = self.min_lr * (self.max_lr / self.min_lr) ** (batch / self.total_steps)
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)
        self.history['lr'].append(lr)
        self.history['loss'].append(logs['loss'])

lr_finder = LRFinderCallback(min_lr=1e-8, max_lr=1, steps=100)

call=EarlyStopping(patience=5,
                  monitor='val_loss',
                  verbose=1,
                  restore_best_weights=True) #그떄까지 가장 좋은 값 return 

check=ModelCheckpoint(filepath='./fl.weights.h5',
                     save_best_only=True,
                     save_weights_only=True,
                     monitor='val_accuracy',
                     verbose=1)

model.fit(train_generator,
          steps_per_epoch=100,
          epochs=30,
          validation_data=validation_generator,
          validation_steps=50,
          callbacks=[lr_finder, call, check],
          verbose=1)

plt.plot(lr_finder.history['lr'], lr_finder.history['loss'])
plt.xscale('log')
plt.xlabel('learning_rate')
plt.ylabel('loss')
plt.show()